In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager

# 设置字体，确保中文可正常显示
font_path = "C:/Windows/Fonts/simhei.ttf"  # 适用于 Windows
# font_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"  # 适用于 Linux
# font_path = "/System/Library/Fonts/Supplemental/Songti.ttc"  # 适用于 Mac

font = font_manager.FontProperties(fname=font_path)
plt.rcParams["font.family"] = font.get_name()
plt.rcParams["axes.unicode_minus"] = False

from ipywidgets import interact, FloatText, FloatSlider
from IPython.display import display
import time

class FlowSimulator:
    def __init__(self, resolution=400):
        self.resolution = resolution
        self.ν = 1.5e-5  # 运动粘度 (m²/s)
        self.x = np.linspace(-3, 3, self.resolution)
        self.y = np.linspace(-3, 3, self.resolution)
        self.X, self.Y = np.meshgrid(self.x, self.y)
        self.U_widget = FloatText(value=5, min=1, max=10, step=0.1, description="流速 U")
        self.a_widget = FloatSlider(value=1.0, min=0.5, max=2.0, step=0.1, description="圆柱半径 a")

    def compute_velocity(self, U, a):
        Z = self.X + 1j * self.Y
        mask = np.abs(Z) > a
        V = np.zeros_like(Z, dtype=np.complex128)
        V[mask] = U * (1 - a**2 / Z[mask]**2)
        return V.real, -V.imag

    def plot_flow(self, U, a):
        u, v = self.compute_velocity(U, a)
        fig, ax = plt.subplots(figsize=(6, 6), facecolor='#87CEEB')
        ax.set_facecolor('#87CEEB')
        ax.streamplot(self.X, self.Y, u, v, color='white', linewidth=0.8, density=2)
        ax.add_patch(plt.Circle((0, 0), a, color='red', alpha=0.5))
        ax.set_xlim([-3, 3])
        ax.set_ylim([-3, 3])
        ax.set_aspect('equal')
        ax.set_title(f'流场可视化 (U={U} m/s, a={a} m)')
        plt.show()

    def run(self):
        interact(lambda U, a: self.plot_flow(U, a), U=self.U_widget, a=self.a_widget)

sim = FlowSimulator()
sim.run()


interactive(children=(FloatText(value=5.0, description='流速 U', step=0.1), FloatSlider(value=1.0, description='…